# Import statements

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import cv2
from PIL import Image

import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


## Getting or Gether Data

In [ ]:
! mkdir ~/.kaggle


In [ ]:
! cp /content/kaggle.json ~/.kaggle

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d biaiscience/dogs-vs-cats

In [ ]:
! unzip /content/dogs-vs-cats.zip

# Data Preprocessing

In [ ]:
path, dirs, files = next(os.walk("./train/train/"))

In [ ]:
path, dirs

In [ ]:
file_count = len(files)

print('Number of Images of Dog and Cats', file_count)

In [ ]:
# printing the image name

print(files[: 10])

In [ ]:
# display the Images

img = plt.imread(f"{path}{files[2]}")

In [ ]:
img.shape

In [ ]:
plt.imshow(img)

In [ ]:
cat_img = plt.imread(f'{path}cat.8070.jpg')

In [ ]:
plt.imshow(cat_img)
plt.axis('off')
plt.show()

In [ ]:
cat_img = cv2.resize(cat_img, (224, 224))

cat_img.shape

In [ ]:
plt.imshow(cat_img)

## Resizing all the images

In [ ]:
cat_list = []
dog_list = []
for item in files:
  name = item.split('.')[0]
  img = plt.imread(f'./train/train/{item}')
  resized_img = cv2.resize(img, (224, 224))
  if name == 'cat':
    cat_list.append(resized_img)
  else:
    dog_list.append(resized_img)


In [ ]:
img = Image.open('/content/train/train/cat.1.jpg')

In [ ]:
img = img.resize((224, 224))

In [ ]:
img = img.convert('RGB')

In [ ]:
img = np.array(img)

In [ ]:
img.shape

## Creating label for image of dog and cats


1.   Cat - 0
2.   Dog - 1



In [ ]:
cats = np.zeros(1000) 
dog = np.zeros(1000) + 1

In [ ]:
label = np.concatenate((cats, dog))

In [ ]:
data_cat = np.array(cat_list[: 1000])

In [ ]:
data_dog = np.array(dog_list[: 1000])

In [ ]:
df = np.concatenate((data_cat, data_dog))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, label, test_size=0.2)

In [ ]:
# Scaling the data

X_train = X_train / 255
X_test = X_test / 255

In [ ]:
X_train.shape

## creating model

In [ ]:
import tensorflow_hub as hub

In [ ]:
mobilenet_model = 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4'

pretrained_model = hub.KerasLayer(mobilenet_model, input_shape=(224,224,3), trainable=False)

In [ ]:
num_of_classes = 2

model = tf.keras.Sequential([
    
    pretrained_model,
    tf.keras.layers.Dense(num_of_classes)

])

model.summary()

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['acc']
)

In [ ]:
model.fit(X_train, y_train, epochs=5)

In [ ]:
score, acc = model.evaluate(X_test, y_test)
print('Test Loss =', score)
print('Test Accuracy =', acc)

In [ ]:
img = X_test[0].reshape(1, 224, 224, 3)

In [ ]:
np.argmax(model.predict(img))

In [ ]:
y_test[0]